# VacationPy Challenge (Geoapify API)
---

## Part #1 - [GeoViews] Create Map Displaying Every City in `city_data_df` DataFrame

### Step #1: Dependencies and Setup

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Use 'Sys' library to add the directory where my api key module file is i.e. point to the root folder
import sys
sys.path.append('../')

# Get my Geoapify API key from the api_keys module file
from api_keys import geoapify_key

### Step #2: Import City Dataset into Pandas DataFrame

In [2]:
# Read the saved City Weather Data from the CSV file where 'City_ID' column is the index column
# Dataset made from the 'WeatherPy' Challenge
city_data_df = pd.read_csv("../output_data/city_weather_data.csv", index_col = "City_ID")

# Display first 5 rows of the City Weather DataFrame (via CSV file)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,isafjordur,66.0755,-23.1240,0.02,93,0,1.03,IS,1698231932
1,blackmans bay,-43.0167,147.3167,7.07,76,45,1.34,AU,1698231934
2,port lincoln,-34.7333,135.8667,11.24,51,14,8.60,AU,1698231934
3,bilibino,68.0546,166.4372,-16.93,99,33,2.02,RU,1698231935
4,new norfolk,-42.7826,147.0587,7.72,82,58,3.49,AU,1698231935


### Step #3: [GeoViews] Create Map Displaying Each City as a Point (Size of Point Determined by Humidity)

In [3]:
%%capture --no-display

# Using Hvplot Library, create a map plot where...
# x = Longitude
# y = Latitude
# Point Size Determined by Humidity
# Each point (city) is coloured differently
map_plot = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    alpha = 0.5,
    color = "City",
    title = "City Humidity on World Map"
)

# Display 'City Humidity on World Map' Map Plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Part #2 - Narrow Down the `city_data_df` DataFrame to Find My Ideal Weather Condition

In [4]:
# Define the criteria masks for the relevant weather attributes

# My Ideal Max Temperature is between 20 and 26 Celsius
temp_mask = (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 26)

# My Ideal Humidity is between 30% and 60%
humid_mask = (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <= 60)

# My Ideal Cloudiness is between 20% and 60%
cloud_mask = (city_data_df["Cloudiness"] >= 20) & (city_data_df["Humidity"] <= 60)


# Combine all masks (ideal weather conditions) into one
ideal_weather_mask = temp_mask & humid_mask & cloud_mask


# Get all rows of cities with weather conditions that fit within my criteria and store in a new DataFrame
best_weather_df = city_data_df[ideal_weather_mask]

# Any rows with NULL values are dropped
best_weather_df = best_weather_df.dropna()

# Display the data
best_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
29,bredasdorp,-34.5322,20.0403,21.92,51,30,4.92,ZA,1698231944
41,hermanus,-34.4187,19.2345,20.71,58,31,4.01,ZA,1698231589
234,asosa,10.0667,34.5333,25.46,55,70,2.01,ET,1698232034
246,gondar,12.6000,37.4667,21.42,59,46,0.95,ET,1698232039
313,mega,4.0500,38.3000,21.93,57,76,5.93,ET,1698232063
342,kulp,38.4972,41.0122,20.73,44,20,1.79,TR,1698232075
368,xuanzhou,30.9525,118.7553,21.98,58,100,1.29,CN,1698232089
373,tobruk,32.0836,23.9764,24.92,55,26,1.87,LY,1698232091
400,kastoria,40.5167,21.2667,25.29,34,100,3.23,GR,1698232104


## Part #3 - Create `hotel_df` DataFrame

In [5]:
# Create a 'Hotel' DataFrame and keep City, Country, Latitude, Longitude and Humidity columns
hotel_df = best_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add 'Hotel Name' column; hotels will be stored using Geoapify API 
hotel_df["Hotel Name"] = None

# Display Hotel DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
29,bredasdorp,ZA,-34.5322,20.0403,51,None
41,hermanus,ZA,-34.4187,19.2345,58,None
234,asosa,ET,10.0667,34.5333,55,None
246,gondar,ET,12.6000,37.4667,59,None
313,mega,ET,4.0500,38.3000,57,None
342,kulp,TR,38.4972,41.0122,44,None
368,xuanzhou,CN,30.9525,118.7553,58,None
373,tobruk,LY,32.0836,23.9764,55,None
400,kastoria,GR,40.5167,21.2667,34,None


## Part #4 - [Geoapify API] For Every City, Find First Hotel Located within 10,000 Metres of My Coordinates

In [6]:
# Set radius as 10,000 metres; search for hotel within 10,000 metres of any coordinates provided
radius = 10000

# Parameter #1: 'accommodation.hotel' (https://www.geoapify.com/places-api)
# Parameter #2: MY Geoapify Key
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Log Commencement of Seraching Hotels
print("-----------------------------------------")
print("-------- Beginning Hotel Search ---------")
print("-----------------------------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the Latitude & Longitude from current row of DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If Hotel is not found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - Nearest Hotel: {hotel_df.loc[index, 'Hotel Name']}")

    
# Log Commencement of Seraching Hotels
print("-----------------------------------------")
print("-------- Hotel Search Complete ----------")
print("-----------------------------------------")
    
# Display Hotel DataFrame w/ Hotel Names
hotel_df

-----------------------------------------
-------- Beginning Hotel Search ---------
-----------------------------------------
bredasdorp - Nearest Hotel: Victoria Hotel
hermanus - Nearest Hotel: Aloe guest house
asosa - Nearest Hotel: Africa Hotel
gondar - Nearest Hotel: Hotel Dashen
mega - Nearest Hotel: No Hotel Found
kulp - Nearest Hotel: No Hotel Found
xuanzhou - Nearest Hotel: 敬亭湖宾馆
tobruk - Nearest Hotel: فندق زهرة البطنان -طبرق
kastoria - Nearest Hotel: Ηδιστον
khorramabad - Nearest Hotel: هتل رنگین کمان
kataba - Nearest Hotel: No Hotel Found
-----------------------------------------
-------- Hotel Search Complete ----------
-----------------------------------------


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
29,bredasdorp,ZA,-34.5322,20.0403,51,Victoria Hotel
41,hermanus,ZA,-34.4187,19.2345,58,Aloe guest house
234,asosa,ET,10.0667,34.5333,55,Africa Hotel
246,gondar,ET,12.6000,37.4667,59,Hotel Dashen
313,mega,ET,4.0500,38.3000,57,No Hotel Found
342,kulp,TR,38.4972,41.0122,44,No Hotel Found
368,xuanzhou,CN,30.9525,118.7553,58,敬亭湖宾馆
373,tobruk,LY,32.0836,23.9764,55,فندق زهرة البطنان -طبرق
400,kastoria,GR,40.5167,21.2667,34,Ηδιστον


## Part #5 - [GeoViews] Use `hotel_df` DataFrame to Create Map; Hotel Name & Country as Additional Information in Hover Message

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"],
    title = "City Humidity on World Map (w/ Hotel Names)"
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)